# Fire Extent Prediction from Historical Weather Data

###### Adam Swan, Jeff Warchall, Omar Younis

## Problem Statement

To help the California Department of Forestry and Fire Protection allocate resources, can we predict the likelihood of fires utilizing historical weather and wildfire data?

### County Location Data

From the California [Open Data Portal](https://data.ca.gov/dataset/ca-geographic-boundaries/resource/b0007416-a325-4777-9295-368ea6b710e6) we obtained a .SHP file of the geographic locations of the counties of California.  A .csv file containing the latitude and longitude of the centers of the counties was then extracted using [MyGeodata Converter](https://mygeodata.cloud/converter/shp-to-csv) stored in the data folder then placed into the ```location``` variable.

In [1]:
import requests
import pandas as pd
import time
from IPython.display import clear_output

In [2]:
counties = pd.read_csv('data/CA_Counties_Location.csv')
location = counties[['NAMELSAD', 'INTPTLAT', 'INTPTLON']]

### Fire Data

The fire data was obtained from the State of California [Geoportal](https://gis.data.ca.gov/), which generated the url used to fetch the data in the cell below.  We obtained all of the fire from 1952 to the present.  These data are described in the data dictionary at the end of Section 02 - Preprocessing.


In [3]:
url = 'https://egis.fire.ca.gov/arcgis/rest/services/FRAP/FirePerimeters_FS/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json'
res = requests.get(url)
data = res.json()

The data is presented in .json format and the features are stored in a dataframe named ```df``` and then exported to a .csv file.

In [4]:
list_of_records = []
list_of_features = data['features'][0]['attributes'].keys()
for index, value in enumerate(data['features']):
    list_of_records.append(data['features'][index]['attributes'].values())
    
df = pd.DataFrame(list_of_records, columns = list_of_features)
df.to_csv('./raw_fires.csv')

### California Weather Data

Weather data was obtained from [World Weather Online](https://www.worldweatheronline.com/developer/) (WWO).  First, using the county latitude and longitude data a list of the centers of each county in California is created.  Then, since the data available on WWO only goes back to July 1, 2008, list containing every year, month, and day in the range 07/01/2008 to 12/31/2020 are created.

In [5]:
lats = list(location['INTPTLAT'])
longs = list(location['INTPTLON'])
years = list(range(2008, 2021))
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
days = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

The WWO API requires a query string to be passed.  These queries are assembled in the following loop to create a unique query for every day in our range of interest.

In [6]:
urls = []
for year in years:
    for i, month in enumerate(months):
        if month > '06' or year > 2008:
            for (j, lat) in enumerate(lats): 
                start_date = str(year)+ '-' + month + '-' + '01'
                end_date = str(year)+ '-' + month + '-' + str(days[i])
                long = longs[j]
                url = 'http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=71ac2af39d93330a7cf2bed42ffc2b22'
                url = url + '&format=json'
                url = url + '&q=' + str(lat) + ',' + str(long)
                url = url + '&tp=24'
                url = url + '&date=' + start_date
                url = url + '&enddate=' + end_date
                urls.append(url)

The list of urls is then passed to the API and the returned .json documents are stored in a list.

In [7]:
data = []
for i, url in enumerate(urls):
    time.sleep(1)
    print("url",url)
    res = requests.get(url)
    data.append(res.json())
    clear_output()
    break
    print(f'{i+1} of 3480 requests complete')
print(data)

[{'data': {'error': [{'msg': 'API key has been disabled.'}]}}]


Finally, we iterate through the .json documents to compile an array of records and save them to a data frame.  This data frame is then exported to the data folder.

In [8]:
list_of_features = list(data[0]['data']['weather'][0].keys())
list_of_features.append('Lat and Long')

list_of_records = []
for i in range(len(data)):
    for dictionary in data[i]['data']['weather']:
        list_of_values = []
        for value in dictionary.values():
            list_of_values.append(value)
        list_of_values.append(data[i]['data']['request'][0]['query'])
        list_of_records.append(list_of_values)

df = pd.DataFrame(list_of_records, columns = list_of_features)
df.to_csv('./clean_daily_weather.csv')

KeyError: 'weather'

avgtempF  totalSnow  humid  wind  precip
0     281.36          0     95  1.79       0
1     280.73          0     95  1.79       0
2     280.06          0     96  0.89       0
3     279.21          0     97  0.89       0
4     279.44          0     98  1.34       0
5     279.11          0     98  0.89       0
6     279.10          0     98  1.34       0
7     278.86          0     98  1.34       0
8     279.08          0     98  0.45       0
9     279.33          0     98  1.79       0
10    278.58          0    100  1.79       0
11    278.64          0    100  1.34       0
12    278.73          0    100  0.89       0
13    279.44          0     98  1.34       0
14    280.54          0     98  0.89       0
15    280.74          0     98  1.34       0
16    281.29          0     98  0.89       0
17    281.97          0     95  0.45       0
18    282.24          0     92  1.34       0
19    282.88          0     93  0.89       0
20    282.82          0     91  0.89       0
21    282.55          0     92  0.89       0
22    282.25          0     90  0.89       0
23    281.56          0     90  0.89       0

In [19]:
for r in res:
    #print(r)
    resdic.update({"avgtempF":r["main"]["temp"] ,"totalSnow":0,"humid":r["main"]["humidity"],"wind":r["wind"]["speed"],"precip":0})
df=pd.DataFrame.from_dict(resdic)
df

{'dt': 1668103200, 'main': {'temp': 281.36, 'feels_like': 280.45, 'pressure': 1031, 'humidity': 95, 'temp_min': 279.58, 'temp_max': 283.56}, 'wind': {'speed': 1.79, 'deg': 0, 'gust': 2.24}, 'clouds': {'all': 40}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}]}
{'dt': 1668106800, 'main': {'temp': 280.73, 'feels_like': 279.73, 'pressure': 1031, 'humidity': 95, 'temp_min': 278.35, 'temp_max': 283}, 'wind': {'speed': 1.79, 'deg': 0, 'gust': 2.24}, 'clouds': {'all': 81}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04n'}]}
{'dt': 1668110400, 'main': {'temp': 280.06, 'feels_like': 280.06, 'pressure': 1031, 'humidity': 96, 'temp_min': 277.79, 'temp_max': 283}, 'wind': {'speed': 0.89, 'deg': 0, 'gust': 2.24}, 'clouds': {'all': 52}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04n'}]}
{'dt': 1668114000, 'main': {'temp': 279.21, 'feels_like': 279.21, 'pressure': 1031, 'humidi

{'avgtempF': 281.56,
 'totalSnow': 0,
 'humid': 90,
 'wind': 0.89,
 'precip': 0,
 'sunHour': ''}